<a href="https://colab.research.google.com/github/WelberFerreira/sistema-de-gerenciamento/blob/master/Extract_chirps_Data_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Importing Data and Acessing the Drive**





In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Import
from sklearn.linear_model import LinearRegression
import pandas as pd
import statsmodels.api as sm
import numpy as np
import sys
from google.colab import drive

In [21]:
# Opening data from CHIRPS
chirps_data = pd.read_csv("/content/drive/MyDrive/CHIRPS/Brutos.txt", delim_whitespace=True)

# **2. Creating a field to store date**

In [22]:
chirps_data['date'] = pd.to_datetime(chirps_data['Data'], format="%d/%m/%Y")

# **3. Creating water year and season groups**

In [23]:
# In this part, the code assign to WY field the correct hydrologic year.
chirps_data['WY'] = chirps_data.date.dt.year.where(chirps_data.date.dt.month < 9, chirps_data.date.dt.year + 1)

In [24]:
# In this part, the code assign to Trim field the correct Seasons.
def assign_wy(row):
    if row.month>=9 and row.month <= 11:
        return('SON')
    elif row.month == 12 or row.month <= 2:
      return('DJF')
    elif row.month >= 3 and row.month <= 5:
      return('MAM')
    else:
        return('JJA')

chirps_data['Trim'] = chirps_data.date.apply(lambda x: assign_wy(x))

In [25]:
# Assigning the index field to field date
chirps_data.set_index('date', inplace = True)


In [26]:
# Retrieving data grouped by season
seasons = chirps_data.groupby(['Id', 'WY', pd.Grouper(key='Trim')])
# Retrieving data grouped by water year
water_years = chirps_data.groupby(['Id', pd.Grouper(key='WY')])

In [ ]:
#display(seasons.precipitation.sum().to_string())

In [ ]:
#display(water_years.precipitation.sum().to_string())

In [29]:
# Saving water year data
sys.path.insert(0,'/content/drive/MyDrive/CHIRPS/')
result = open("/content/drive/MyDrive/CHIRPS/TotalAnual.csv", "w+")


# **4. Generating csv file to use in the trending test**

In [38]:
# Looping through the data in order to transform the data to trending analysis
# format. Using just Water Year
rainfall_water_year = water_years.precipitation.sum() # Total annual rainfall

years = chirps_data['WY'].unique()
n_years = len(years)
codes = chirps_data['Id'].unique()


for i in range(len(codes)): # Looping through the IDs
  if i == 0:
    result.write("Years")
    result.write(";")
    result.write(str(codes[i]))
    result.write(";")

  elif i == (len(codes) - 1):
    result.write(str(codes[i]))
    result.write("\n")

  else:
    result.write(str(codes[i]))
    result.write(";")


for i in range(n_years): # Looping through the years
  k = i
  #print(years[i])
  result.write(str(years[i]))
  result.write(";")
  for j in range(len(codes)): # Looping through the IDs

    try:
      result.write(str(round(rainfall_water_year.iloc[k],2)))
      #print(simple_data.iloc[k])

    except:
      result.write(str("Null"))

    if j != (len(codes) - 1):
      result.write(";")
    k+= (n_years)

  result.write("\n")


result.close()

Id   WY  
1    1981     780.229368
     1982    1839.811655
     1983    1675.899403
     1984    1572.938999
     1985    1354.237828
                ...     
240  2017    1218.696563
     2018    1419.372295
     2019    1663.816262
     2020    1518.656443
     2021     579.321007
Name: precipitation, Length: 9840, dtype: float64


In [ ]:
# Looping through the data in order to transform the data to trending analysis
# format. Using Seasons

seasons_names = chirps_data['Trim'].unique()

for season in seasons_names:
    file_name = "/content/drive/MyDrive/CHIRPS/Results/Total_" + season + ".csv"
    result_season = open(file_name, "w+")

    season_data = chirps_data[chirps_data["Trim"] == season]
    data_temp = season_data.groupby(['Id', 'WY', pd.Grouper(key='Trim')])
    #data_temp = season_data.groupby(['Id', pd.Grouper(key='WY')])
    seasons_data_total = data_temp.precipitation.sum()

    # seasons_data_total.index Contains the index (code, year, season)
    # in order to get number of years we need to use 1 in brackets
    data_points = len(seasons_data_total.index.unique(1))# number of years

    for i in range(len(codes)): # Looping through the IDs in order to create
                                # the header
      if i == 0:
        result_season.write("Years")
        result_season.write(";")
        result_season.write(str(codes[i]))
        result_season.write(";")

      elif i == (len(codes) - 1):
        result_season.write(str(codes[i]))
        result_season.write("\n")

      else:
        result_season.write(str(codes[i]))
        result_season.write(";")

    for n in range(data_points): # Looping through the years
      k = n

      # in order to get the current year we need to use 1 in brackets
      result_season.write(str(seasons_data_total.index[n][1]))

      result_season.write(";")

      for j in range(len(codes)): # Looping through the IDs
        result_season.write(str(round(seasons_data_total.iloc[k],2)))

        if j != (len(codes) - 1):
          result_season.write(";")

        k+= (data_points) # jump to the next year

      result_season.write("\n")


    result_season.close()